## 1. 归并排序O(N*logN)

In [62]:
def merge(arr, L, M, R):
    helpArr = [None] * (R - L + 1) 
    index = 0
    P1 = L
    P2 = M+1
    while P1 <= M and P2 <= R:
        if arr[P1] <= arr[P2]:
            helpArr[index] = arr[P1]
            P1 += 1
        else:
            helpArr[index] = arr[P2]
            P2 += 1
        index += 1
    while P1 <= M:
        helpArr[index] = arr[P1]
        P1 += 1
        index += 1
    while P2 <= R:
        helpArr[index] = arr[P2]
        P2 += 1
        index += 1
    for i in range(R-L+1):
        arr[L+i] = helpArr[i]
    return

# 递归版    
def process(arr, L, R):
    if L == R:
        return
    mid = L + ((R-L)>>1)
    process(arr, L, mid)
    process(arr, mid+1, R)
    merge(arr, L, mid, R)
    return
# T(N) = 2*T(N/2) + O(N) ==> O(logN*N)
# 比较行为相比O(N^2)的排序算法，并没有浪费
def mergeSort(arr):
    if not arr or len(arr) == 1:
        return
    process(arr, 0, len(arr) - 1)
    return 

# 迭代版
def mergeSort2(arr):
    if not arr or len(arr) == 1:
        return
    N = len(arr)
    mergeSize = 1
    while mergeSize < N:
        L = 0
        while L < N - 1: #L不是最后一个数
            # L...M 左组
            M = L + mergeSize - 1
            if M >= N - 1:
                break
            # M+1...R 右组
            R = min(M + mergeSize, N-1)
            merge(arr, L, M, R)
            L = R + 1
        if mergeSize > (N // 2):
            break
        mergeSize <<= 1  # 等价于 1 << mergeSize
            

In [63]:
arr=[6,3,2,1,6,7]
arr2 = arr[:]
mergeSort(arr)
mergeSort2(arr2)
print(arr)
print(arr2)

[1, 2, 3, 6, 6, 7]
[1, 2, 3, 6, 6, 7]


## 2.面试题

### 2.1 小和问题
每个数左边小于这个数的和为小和，一个数组所有的小和累加是多少即为小和问题。
* 数组 = [6,3,2,1,6,7]
* 小和 = [0,0,0,0,6,18]
* 返回 = 6 + 18 = 24

In [99]:
#思路： 原题可转换为 一个数的右边有几个比当前数大，即产生sum(几个*当前数)的小和
#mergeSort的merge过程中，
#左值(indexLeft) < 右值(indexRight)时， 产生小和，且产生(R-indexRight+1)个左值；
#左值            >= 右值时，直接拷贝。相对需要先拷贝右边，因为这样才能让左值正确遇到第一个比他大时，通过R-indexRight算出多少个比他大
'''
只盯着x
x,5,6,3,2,3,4,5
  ^            第一次merge，x考虑5是否比他大
    ^ ^        第二次merge，x考虑6,3是否比他大
        ^ ^ ^ ^第三次merge，x考虑2,3,4,5是否比他大
当x为数组内的每个数，所有数的小和都被考虑了！
'''
def merge(arr, L, M, R):
    helpArr = [None] * (R - L + 1) 
    index = 0
    P1 = L
    P2 = M+1
    #新加: 相比原merge
    res = 0
    print(L,R)
    while P1 <= M and P2 <= R:
        if arr[P1] < arr[P2]:
            #新加: 相比原merge多一行res
            print((R - P2 + 1), "*", arr[P1])  
            res += ((R - P2 + 1) * arr[P1])            
            helpArr[index] = arr[P1]
            P1 += 1
        else:
            helpArr[index] = arr[P2]
            P2 += 1
        index += 1
    while P1 <= M:
        helpArr[index] = arr[P1]
        P1 += 1
        index += 1
    while P2 <= R:
        helpArr[index] = arr[P2]
        P2 += 1
        index += 1
    for i in range(R-L+1):
        arr[L+i] = helpArr[i]
    return res

#既L...R排好序，又要求小和返回
def process(arr, L, R):
    if L == R:
        return 0
    mid = L + ((R-L)>>1)
    return process(arr, L, mid) + process(arr, mid+1, R) + merge(arr, L, mid, R)

def smallSum(arr):
    if not arr or len(arr) < 2:
        return 0
    return process(arr, 0, len(arr)-1)

In [ ]:
# 注意 如果需要返回个数，那么sort过程可以加速；但如果是需要遍历出所有小和对，则无法加速，不排序无影响

### 2.2 逆序对
每个数与右侧比自己小的数，可以形成逆序对
* 数组 = [6,3,2,1,6,7]
* 逆序对 = [(6,3),(6,2),(6,1),(3,2),(3,1),(2,1)]
* 返回 = 6

In [119]:
# 思路： mergeSort过程中，左侧数比右侧数大，产生(R-indexRight+1)个逆序对。
# 注意： 要从右往左拷贝，谁大拷贝谁，且相等时先拷贝右侧，因为要保证左侧遇到第一个比自己小的后才能拷贝

def merge(arr, L, M, R):
    helpArr = [None] * (R - L + 1) 
    index = (R - L)
    P1 = M
    P2 = R
    res = 0
    #新加: 相比原merge
    while P1 >= L and P2 >= M+1:
        if arr[P1] > arr[P2]:
            #新加: 相比原merge
            res += (P2-M)
            helpArr[index] = arr[P1]
            P1 -= 1
        else:
            helpArr[index] = arr[P2]
            P2 -= 1
        index -= 1
    while P1 >= L:
        helpArr[index] = arr[P1]
        P1 -= 1
        index -=1
    while P2 >= M+1:
        helpArr[index] = arr[P2]
        P2 -=1
        index -= 1
    for i in range(R-L+1):
        arr[L+i] = helpArr[i]
    return res
def process(arr, L, R):
    if L == R:
        return 0
    mid = L + ((R-L)>>1)
    return process(arr, L, mid)+process(arr, mid+1, R)+merge(arr, L, mid, R)

def reversePair(arr):
    if not arr or len(arr) < 2:
        return 0
    ans = process(arr, 0, len(arr)-1)
    return ans


In [120]:
reversePair([6,3,2,1,6,7])

6

### 2.3 左数>右数*2的(左,右)对数
* 数组 = [6,3,2,1,6,7]
* 组对 = [(6,2),(6,1),(3,1)]
* 返回 = 3


In [1]:
#思路： 两侧数组保持有序，左侧指针指第一个数，右侧指针放在-1(0之前)；对于每个左数，右侧尝试往右划，划不动时计算对数
def merge(arr, L, M, R):
    res = 0
    w2 = M + 1
    #w2表示 window[w1, w2)， 到不了w2
    for w1 in range(L, M+1):
        while w2 <= R and arr[w1] > (arr[w2]*2): #w2不回退技巧！O(N)
            w2 += 1
        #对于w1，划到了可以划的最右侧
        res += (w2 - M - 1)
    helpArr = [None] * (R - L + 1)
    index = 0
    P1 = L
    P2 = M+1
    while P1 <= M and P2 <= R:
        if arr[P1] <= arr[P2]:
            helpArr[index] = arr[P1]
            P1 += 1
        else:
            helpArr[index] = arr[P2]
            P2 += 1
        index += 1
    while P1 <= M:
        helpArr[index] = arr[P1]
        P1 += 1
        index += 1
    while P2 <= R:
        helpArr[index] = arr[P2]
        P2 += 1
        index += 1
    for i in range(R-L+1):
        arr[L+i] = helpArr[i]
    return res

def process(arr, L, R):
    if L == R:
        return 0
    mid = L + ((R - L)>>1)
    return process(arr, L, mid) + process(arr, mid+1, R) + merge(arr, L, mid, R)


def biggerThanTwice(arr):
    if not arr or len(arr) < 2:
        return 0
    return process(arr, 0, len(arr)-1)

In [124]:
arr = [6,3,2,1,6,7]
biggerThanTwice(arr)

3

In [125]:
arr = [1,3,9,13,1,1,1,4,5]
biggerThanTwice(arr)

12